In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [ ]:
import multiprocessing
from torch import autograd
from fastai.conv_learner import *
from fastai.transforms import TfmType
from fasterai.transforms import *
from fasterai.images import *
from fasterai.dataset import *
from fasterai.visualize import *
from fasterai.callbacks import *
from fasterai.loss import *
from fasterai.modules import *
from fasterai.training import *
from fasterai.generators import *
from fastai.torch_imports import *
from pathlib import Path
from itertools import repeat
import tensorboardX
torch.cuda.set_device(1)
plt.style.use('dark_background')
torch.backends.cudnn.benchmark=True


In [ ]:
IMAGENET = Path('data/imagenet/ILSVRC/Data/CLS-LOC/train')
proj_id = 'bwc_rc'
TENSORBOARD_PATH = Path('data/tensorboard/' + proj_id)

#gpath = IMAGENET.parent/(proj_id + '_gen_64.h5')
#dpath = IMAGENET.parent/(proj_id + '_critic_64.h5')

c_lr=5e-4
c_lrs = np.array([c_lr,c_lr,c_lr])

g_lr=c_lr/5
g_lrs = np.array([g_lr/1000,g_lr/100,g_lr])

keep_pcts=[0.25,0.25]
gen_freeze_tos=[-1,0]
lrs_unfreeze_factor=1.0
x_tfms = [BlackAndWhiteTransform()]
extra_aug_tfms = [RandomLighting(0.1, 0.1, tfm_y=TfmType.PIXEL)]
torch.backends.cudnn.benchmark=True

## Training

In [ ]:
netG = Unet34(nf_factor=2).cuda()
#netGVis = ModelVisualizationHook(TENSORBOARD_PATH, netG, 'netG')
#load_model(netG, gpath)

netD = DCCritic(ni=3, nf=512).cuda()
#netDVis = ModelVisualizationHook(TENSORBOARD_PATH, netD, 'netD')
#load_model(netD, dpath)

In [ ]:
trainer = GANTrainer(netD=netD, netG=netG, genloss_fns=[FeatureLoss(multiplier=1e2)])
trainerVis = GANVisualizationHook(TENSORBOARD_PATH, trainer, 'trainer', jupyter=False, visual_iters=100)

In [ ]:
scheds=[]

scheds.extend(GANTrainSchedule.generate_schedules(szs=[64,64], bss=[32,32], path=IMAGENET, x_tfms=x_tfms, extra_aug_tfms=extra_aug_tfms,keep_pcts=keep_pcts, 
    save_base_name=proj_id, c_lrs=c_lrs, g_lrs=g_lrs, lrs_unfreeze_factor=lrs_unfreeze_factor, gen_freeze_tos=gen_freeze_tos))

#unshock
scheds.extend(GANTrainSchedule.generate_schedules(szs=[96], bss=[16], path=IMAGENET, x_tfms=x_tfms, extra_aug_tfms=extra_aug_tfms, keep_pcts=[0.1], 
    save_base_name=proj_id, c_lrs=c_lrs/20, g_lrs=g_lrs/20, lrs_unfreeze_factor=lrs_unfreeze_factor, gen_freeze_tos=[-1]))

scheds.extend(GANTrainSchedule.generate_schedules(szs=[96,96], bss=[16,16], path=IMAGENET, x_tfms=x_tfms, extra_aug_tfms=extra_aug_tfms, keep_pcts=keep_pcts, 
    save_base_name=proj_id, c_lrs=c_lrs/2, g_lrs=g_lrs/2, lrs_unfreeze_factor=lrs_unfreeze_factor, gen_freeze_tos=gen_freeze_tos))


#unshock
scheds.extend(GANTrainSchedule.generate_schedules(szs=[128], bss=[12], path=IMAGENET, x_tfms=x_tfms, extra_aug_tfms=extra_aug_tfms, keep_pcts=[0.1], 
    save_base_name=proj_id, c_lrs=c_lrs/30, g_lrs=g_lrs/30, lrs_unfreeze_factor=lrs_unfreeze_factor, gen_freeze_tos=[-1]))

scheds.extend(GANTrainSchedule.generate_schedules(szs=[128,128], bss=[12,12], path=IMAGENET, x_tfms=x_tfms, extra_aug_tfms=extra_aug_tfms, keep_pcts=keep_pcts, 
    save_base_name=proj_id, c_lrs=c_lrs/3, g_lrs=g_lrs/3, lrs_unfreeze_factor=lrs_unfreeze_factor, gen_freeze_tos=gen_freeze_tos))


#unshock
scheds.extend(GANTrainSchedule.generate_schedules(szs=[192], bss=[4], path=IMAGENET, x_tfms=x_tfms, extra_aug_tfms=extra_aug_tfms, keep_pcts=[0.1], 
    save_base_name=proj_id, c_lrs=c_lrs/80, g_lrs=g_lrs/80, lrs_unfreeze_factor=lrs_unfreeze_factor, gen_freeze_tos=[-1]))

scheds.extend(GANTrainSchedule.generate_schedules(szs=[192], bss=[4], path=IMAGENET, x_tfms=x_tfms, extra_aug_tfms=extra_aug_tfms, keep_pcts=[0.25], 
    save_base_name=proj_id, c_lrs=c_lrs/8, g_lrs=g_lrs/8, lrs_unfreeze_factor=lrs_unfreeze_factor, gen_freeze_tos=[-1]))

scheds.extend(GANTrainSchedule.generate_schedules(szs=[192], bss=[4], path=IMAGENET, x_tfms=x_tfms, extra_aug_tfms=extra_aug_tfms, keep_pcts=[0.25], 
    save_base_name=proj_id, c_lrs=c_lrs/16, g_lrs=g_lrs/16, lrs_unfreeze_factor=lrs_unfreeze_factor, gen_freeze_tos=[0]))



#unshock
scheds.extend(GANTrainSchedule.generate_schedules(szs=[256], bss=[2], path=IMAGENET, x_tfms=x_tfms, extra_aug_tfms=extra_aug_tfms, keep_pcts=[0.1], 
    save_base_name=proj_id, c_lrs=c_lrs/160, g_lrs=g_lrs/160, lrs_unfreeze_factor=lrs_unfreeze_factor, gen_freeze_tos=[-1]))

scheds.extend(GANTrainSchedule.generate_schedules(szs=[256], bss=[2], path=IMAGENET, x_tfms=x_tfms, extra_aug_tfms=extra_aug_tfms, keep_pcts=[0.25], 
    save_base_name=proj_id, c_lrs=c_lrs/16, g_lrs=g_lrs/16, lrs_unfreeze_factor=lrs_unfreeze_factor, gen_freeze_tos=[-1]))

scheds.extend(GANTrainSchedule.generate_schedules(szs=[256], bss=[2], path=IMAGENET, x_tfms=x_tfms, extra_aug_tfms=extra_aug_tfms, keep_pcts=[0.25], 
    save_base_name=proj_id, c_lrs=c_lrs/32, g_lrs=g_lrs/32, lrs_unfreeze_factor=lrs_unfreeze_factor, gen_freeze_tos=[0]))


In [ ]:
trainer.train(scheds=scheds)

 24%|██▎       | 2280/9637 [21:43<1:01:13,  2.00it/s]
HingeLoss 1.920006275177002; RScore 0.9806321263313293; FScore 0.9393741488456726; GAddlLoss [2.86979]; Iters: 1140; GCost: 0.033145513385534286;
 24%|██▍       | 2300/9637 [21:53<1:00:31,  2.02it/s]
HingeLoss 1.7997559309005737; RScore 1.1856446266174316; FScore 0.6141113042831421; GAddlLoss [2.85213]; Iters: 1150; GCost: 0.47137489914894104;
 24%|██▍       | 2320/9637 [22:04<1:00:26,  2.02it/s]
HingeLoss 1.593624472618103; RScore 1.3018876314163208; FScore 0.29173681139945984; GAddlLoss [3.09584]; Iters: 1160; GCost: 0.35161638259887695;
 24%|██▍       | 2340/9637 [22:14<1:00:34,  2.01it/s]
HingeLoss 1.902028203010559; RScore 0.934773325920105; FScore 0.9672548770904541; GAddlLoss [3.21826]; Iters: 1170; GCost: 0.6368169188499451;
 24%|██▍       | 2360/9637 [22:25<1:00:15,  2.01it/s]
HingeLoss 1.8702163696289062; RScore 1.2792446613311768; FScore 0.5909717679023743; GAddlLoss [2.82861]; Iters: 1180; GCost: 0.5358245968818665;
 25%